【说明】  
pandas是基于NumPy数组构建的，特别是基于数组的函数和不使用for循环的数据处理。

虽然pandas采用了大量的NumPy编码风格，但二者最大的不同是pandas是专门为处理表格和混杂数据设计的。而NumPy更适合处理统一的数值数组数据。

In [1]:
import pandas as pd

## 5.1 pandas的数据结构介绍
要使用pandas，你首先就得熟悉它的两个主要数据结构：Series和DataFrame。

### Series
Series是一种类似于一维数组的对象，它由一组数据（各种NumPy数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的Series：

In [2]:
obj = pd.Series([4,7,-5,3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个0到N-1（N为数据的长度）的整数型索引。你可以通过Series 的values和index属性获取其数组表示形式和索引对象：

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
import numpy as np
arr = np.array([4,7,-5,3])

In [6]:
arr

array([ 4,  7, -5,  3])

通常，我们希望所创建的Series带有一个可以对各个数据点进行标记的索引：

In [7]:
obj2 = pd.Series([4,7,-5,3], index = ['d','b','a','c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [8]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

与普通NumPy数组相比，你可以通过索引的方式选取Series中的单个或一组值：

In [9]:
obj2['a']

-5

In [10]:
obj2['d']

4

In [11]:
obj2['d'] = 6

In [12]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [13]:
obj2['a']

-5

In [14]:
obj2['d'] = 6

In [15]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

['c', 'a', 'd']是索引列表，即使它包含的是字符串而不是整数。

使用NumPy函数或类似NumPy的运算（如根据布尔型数组进行过滤、标量乘法、应用数学函数等）都会保留索引值的链接：

In [16]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [17]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [18]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

还可以将Series看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [19]:
'b' in obj2

True

In [20]:
'e' in obj2

False

In [21]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

如果只传入一个字典，则结果Series中的索引就是原字典的键（有序排列）。你可以传入排好序的字典的键以改变顺序：

In [22]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

在这个例子中，sdata中跟states索引相匹配的那3个值会被找出来并放到相应的位置上，但由于"California"所对应的sdata值找不到，所以其结果就为NaN（即“非数字”（not a number），在pandas中，它用于表示缺失或NA值）。因为‘Utah’不在states中，它被从结果中除去。

我将使用缺失（missing）或NA表示缺失数据。pandas的isnull和notnull函数可用于检测缺失数据：

In [23]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [24]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [25]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于许多应用而言，Series最重要的一个功能是，它会根据运算的索引标签自动对齐数据：

In [26]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [27]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [28]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

数据对齐功能将在后面详细讲解。如果你使用过数据库，你可以认为是类似join的操作。

Series对象本身及其索引都有一个name属性，该属性跟pandas其他的关键功能关系非常密切：

In [29]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

Series的索引可以通过赋值的方式就地修改：

In [30]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [31]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [32]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame
DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）。DataFrame既有行索引也有列索引，它可以被看做由Series组成的字典（共用同一个索引）。DataFrame中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）.

In [33]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}

In [34]:
frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


对于特别大的DataFrame，head方法会选取前五行：

In [35]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


如果指定了列序列，则DataFrame的列就会按照指定顺序进行排列：

In [36]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传入的列在数据中找不到，就会在结果中产生缺失值：

In [37]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'], index = ['one', 'two', 'three', 
                                                                      'four', 'five', 'six'])

通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series：

In [38]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [39]:
frame2.state
# 跟 frame2['state'] 效果一样

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

注意，返回的Series拥有原DataFrame相同的索引，且其name属性也已经被相应地设置好了。  

行也可以通过位置或名称的方式进行获取

In [40]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [41]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

In [42]:
frame2['debt'] = 16.5

In [43]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [44]:
frame2['debe'] = np.arange(6.0)

In [45]:
frame2

,year,state,pop,debt,debe
one,2000,Ohio,1.5,16.5,0.0
two,2001,Ohio,1.7,16.5,1.0
three,2002,Ohio,3.6,16.5,2.0
four,2001,Nevada,2.4,16.5,3.0
five,2002,Nevada,2.9,16.5,4.0
six,2003,Nevada,3.2,16.5,5.0


In [46]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debe'] = val

In [47]:
frame2

,year,state,pop,debt,debe
one,2000,Ohio,1.5,16.5,NaN
two,2001,Ohio,1.7,16.5,-1.2
three,2002,Ohio,3.6,16.5,NaN
four,2001,Nevada,2.4,16.5,-1.5
five,2002,Nevada,2.9,16.5,-1.7
six,2003,Nevada,3.2,16.5,NaN


为不存在的列赋值会创建出一个新列。关键字del用于删除列。

作为del的例子，我先添加一个新的布尔值的列，state是否为'Ohio'：

In [48]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,debe,eastern
one,2000,Ohio,1.5,16.5,NaN,True
two,2001,Ohio,1.7,16.5,-1.2,True
three,2002,Ohio,3.6,16.5,NaN,True
four,2001,Nevada,2.4,16.5,-1.5,False
five,2002,Nevada,2.9,16.5,-1.7,False
six,2003,Nevada,3.2,16.5,NaN,False


del frame2['eastern']
frame2

In [49]:
frame2.columns

Index(['year', 'state', 'pop', 'debt', 'debe', 'eastern'], dtype='object')

另一种常见的数据形式是嵌套字典：

In [50]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

如果嵌套字典传给DataFrame，pandas就会被解释为：**外层字典的键作为列，内层键则作为行索引：**

In [51]:
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


你也可以使用类似NumPy数组的方法，对DataFrame进行转置（交换行和列）：

In [52]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


In [53]:
pop

{'Nevada': {2001: 2.4, 2002: 2.9}, 'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不会这样：

In [54]:
pd.DataFrame(pop, index = [2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


由Series组成的字典差不多也是一样的用法：

In [55]:
pdata = {'Ohio': frame3['Ohio'][:-1],
          'Nevada': frame3['Nevada'][:2]}

In [56]:
frame3['Ohio'][:-1]

2001    1.7
2002    3.6
Name: Ohio, dtype: float64

In [57]:
import numpy as np
arr = np.array([2,4,6,8,10])
arr

array([ 2,  4,  6,  8, 10])

In [58]:
arr[:4] # 表示从0到索引4的值（不包括4）

array([2, 4, 6, 8])

In [59]:
arr[4:] # 表示第四个后面的所有值

array([10])

In [60]:
pd.DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


In [61]:
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


![jupyter](./assets/Snipaste_2024-04-06_10-37-49.JPG)

In [62]:
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [63]:
frame3.index.name = 'year'
frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [64]:
# 跟Series一样，values属性也会以二维ndarray的形式返回DataFrame中的数据：
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

In [65]:
# 如果DataFrame各列的数据类型不同，则值数组的dtype就会选用能兼容所有列的数据类型：
frame2.values

array([[2000, 'Ohio', 1.5, 16.5, nan, True],
       [2001, 'Ohio', 1.7, 16.5, -1.2, True],
       [2002, 'Ohio', 3.6, 16.5, nan, True],
       [2001, 'Nevada', 2.4, 16.5, -1.5, False],
       [2002, 'Nevada', 2.9, 16.5, -1.7, False],
       [2003, 'Nevada', 3.2, 16.5, nan, False]], dtype=object)

### 索引对象
pandas的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index：

In [66]:
obj = pd.Series(range(3), index=['a', 'b', 'c']) 

In [67]:
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [68]:
index[1:]

Index(['b', 'c'], dtype='object')

In [69]:
index[:1]

Index(['a'], dtype='object')

**Index对象是不可变的，因此用户不能对其进行修改：**

In [70]:
# 如下操作会报错
# index[1] = 'd'

不可变可以使Index对象在多个数据结构之间安全共享：

In [71]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [72]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [73]:
obj2.index is labels

True

> 注意：虽然用户不需要经常使用Index的功能，但是因为一些操作会生成包含被索引化的数据，理解它们的工作原理是很重要的。

除了类似于数组，Index的功能也类似一个固定大小的集合：

In [74]:
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [75]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [76]:
frame3.columns.name

'state'

'Ohio' in frame3.columns

In [77]:
frame3.index

Int64Index([2001, 2002, 2000], dtype='int64', name='year')

In [78]:
2003 in frame3.index

False

与python的集合不同，**pandas的Index可以包含重复的标签：**

In [79]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

In [80]:
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

选择重复的标签，会显示所有的结果。  

每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含的数据的常见问题。表5-2列出了这些函数。

![Index的方法和属性](./assets/Snipaste_2024-04-07_21-19-37.JPG)

## 5.2 基本功能
本节中，我将介绍操作Series和DataFrame中的数据的基本手段。

### 重新索引
pandas对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。看下面的例子：

In [81]:
obj = pd.Series([4.5, 7.2,-5.3, 3.6], index = ['d', 'b', 'a', 'c'])

In [82]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

用该Series的reindex将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：

In [83]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2, obj

(a   -5.3
 b    7.2
 c    3.6
 d    4.5
 e    NaN
 dtype: float64,
 d    4.5
 b    7.2
 a   -5.3
 c    3.6
 dtype: float64)

In [84]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])

obj3.reindex(range(7), method='ffill')


0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
6    yellow
dtype: object

借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

In [85]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)),
                     index = ['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California']
)
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [86]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [87]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [88]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)


,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


![](./assets/Snipaste_2024-04-08_21-28-21.JPG)

### 丢弃指定轴上的项
丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象：

In [89]:
obj = pd.Series(np.arange(5.), index = ['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [90]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [91]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [92]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
               index=['Ohio', 'Colorado', 'Utah', 'New York'],
               columns=['one', 'two', 'three', 'four']     
)
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


用标签序列调用drop会从行标签（axis 0）删除值：

In [93]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


通过传递axis=1或axis='columns'可以删除列的值：

In [94]:
data.drop('two',axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [95]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


许多函数，如drop，会修改Series或DataFrame的大小或形状，可以就地修改对象，不会返回新的对象：

In [96]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

小心使用inplace，它会销毁所有被删除的数据。

### 索引、选取和过滤
Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。下面是几个例子：

In [97]:
obj = pd.Series(np.arange(4.0), index = ['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [98]:
obj['b']

1.0

In [99]:
obj[1]

1.0

In [100]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [101]:
obj[['b','a','d']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [102]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [103]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

利用标签的切片运算与普通的Python切片运算不同，其末端是包含的：

In [104]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

用切片可以对Series的相应部分进行设置：

In [105]:
obj['b','c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [106]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four']
)
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [107]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [108]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取数据：

In [109]:
data[:2] # 前2行

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [110]:
data[data['three'] > 5] # data['three'] > 5 所在的行

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


选取行的语法data[:2]十分方便。向[ ]传递单一的元素或列表，就可选择列。

另一种用法是通过布尔型DataFrame（比如下面这个由标量比较运算得出的）进行索引：

In [111]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [112]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### 用loc和iloc进行选取
对于DataFrame的行的标签索引，我引入了特殊的标签运算符loc和iloc。它们可以让你用类似NumPy的标记，使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。

作为一个初步示例，让我们通过标签选择一行和多列：

In [113]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [114]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

然后用iloc和整数进行选取：

In [115]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [116]:
data.iloc[2, [3,0,1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [117]:
data.iloc[[1,2], [3,0,1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


这两个索引函数也适用于一个标签或多个标签的切片：

In [118]:
data.loc[:'Utah', 'two'] # 包含了 Utah 行

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [119]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [120]:
data.iloc[:, :3]

,one,two,three
Ohio,0,0,0
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [121]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


所以，在pandas中，有多个方法可以选取和重新组合数据。

![](./assets/Snipaste_2024-04-09_08-19-01.JPG)

### 整数索引
处理整数索引的pandas对象常常难住新手，因为它与Python内置的列表和元组的索引语法不同。例如，你可能不认为下面的代码会出错：

In [122]:
ser = pd.Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

这里，pandas可以勉强进行整数索引，但是会导致小bug。我们有包含0,1,2的索引，但是引入用户想要的东西（基于标签或位置的索引）很难：

In [123]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [124]:
ser2[-1]

2.0

为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：

In [125]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [126]:
ser[:1]

0    0.0
dtype: float64

In [127]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [128]:
ser.iloc[:1]

0    0.0
dtype: float64

In [ ]:
### 算术运算和数据对齐
pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例子：